In [15]:
DATA_DIR = 'data/'
RESULT_DIR = 'results/original_vs_updated/'
import os
# Create the directories
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

In [12]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("dl2-2024/dl-2024")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv(DATA_DIR + "project.csv")

In [10]:
from tabulate import tabulate

# Read the CSV file
runs_df = pd.read_csv(DATA_DIR + "project.csv")

# Extract and flatten the summary and config columns
summary_df = pd.json_normalize(runs_df['summary'].apply(eval))
config_df = pd.json_normalize(runs_df['config'].apply(eval))

# Combine the flattened summary and config data
results_df = pd.concat([runs_df[['name']], config_df, summary_df], axis=1)

# Filter results based on `softmax` and `use_underscore` settings
original_code = results_df[(results_df['softmax'] == True) & (results_df['use_underscore'] == True)]
updated_code = results_df[(results_df['softmax'] == False) & (results_df['use_underscore'] == False)]

# Function to get the final_top1_acc for each dataset
def get_final_top1_acc(df, dataset_name):
    result = df[df['dataset_name'] == dataset_name]['final_top1_acc']
    return result.iloc[0] if not result.empty else None

# Create a summary DataFrame
summary_data = {
    "Method": ["Original Code - lambda equitune", "Updated Code - lambda equitune"],
    "CIFAR100": [
        get_final_top1_acc(original_code, "CIFAR100"),
        get_final_top1_acc(updated_code, "CIFAR100")
    ],
    "MNIST": [
        get_final_top1_acc(original_code, "MNIST"),
        get_final_top1_acc(updated_code, "MNIST")
    ],
    "ISIC2018": [
        get_final_top1_acc(original_code, "ISIC2018"),
        get_final_top1_acc(updated_code, "ISIC2018")
    ]
}

summary_df = pd.DataFrame(summary_data)

# # Save the summary table to a new CSV file
# summary_df.to_csv(RESULT_DIR + "final_top1_accuracy_summary.csv", index=False)

# Convert the dataframe to a markdown table
markdown_table = tabulate(summary_df, headers='keys', tablefmt='pipe')

# Save the markdown table to a file
with open(RESULT_DIR + "final_top1_accuracy_summary.md", "w") as file:
    file.write(markdown_table)

# Display the markdown table
print(markdown_table)


|    | Method                          |   CIFAR100 |   MNIST |   ISIC2018 |
|---:|:--------------------------------|-----------:|--------:|-----------:|
|  0 | Original Code - lambda equitune |         97 |      65 |         12 |
|  1 | Updated Code - lambda equitune  |         25 |       7 |         65 |


In [17]:
import pandas as pd
from tabulate import tabulate

# Read the CSV file
runs_df = pd.read_csv(DATA_DIR + "project.csv")

# Extract and flatten the summary and config columns
summary_df = pd.json_normalize(runs_df['summary'].apply(eval))
config_df = pd.json_normalize(runs_df['config'].apply(eval))

# Combine the flattened summary and config data
results_df = pd.concat([runs_df[['name']], config_df, summary_df], axis=1)

# Filter results based on `softmax` and `use_underscore` settings
original_code = results_df[(results_df['softmax'] == True) & (results_df['use_underscore'] == True)]
updated_code = results_df[(results_df['softmax'] == False) & (results_df['use_underscore'] == False)]

# Function to get the final_top1_acc for each dataset, model, and transformation combination
def get_final_top1_acc(df, dataset_name, model_name, transformation):
    result = df[(df['dataset_name'] == dataset_name) & 
                (df['model_name'] == model_name) & 
                (df['data_transformations'] == transformation)]['final_top1_acc']
    return result.iloc[0] if not result.empty else None

# Create a summary DataFrame for each combination
architectures = ['RN50', 'RN101']  # Add more architectures if needed
transformations = ['flip', 'rot90']
datasets = ['CIFAR100']  # Add more datasets if needed

summary_data = {
    "Architecture-Transformation": [],
    "Original Code Top1-Acc": [],
    "Updated Code Top1-Acc": []
}

for architecture in architectures:
    for transformation in transformations:
        for dataset in datasets:
            combination = f"{architecture} {transformation} {dataset}"
            summary_data["Architecture-Transformation"].append(combination)
            summary_data["Original Code Top1-Acc"].append(get_final_top1_acc(original_code, dataset, architecture, transformation))
            summary_data["Updated Code Top1-Acc"].append(get_final_top1_acc(updated_code, dataset, architecture, transformation))

summary_df = pd.DataFrame(summary_data)

# Save the summary table to a new CSV file
# summary_df.to_csv(RESULT_DIR + "final_top1_accuracy_summary.csv", index=False)

# Convert the dataframe to a markdown table
markdown_table = tabulate(summary_df, headers='keys', tablefmt='pipe')

# # Save the markdown table to a file
# with open(RESULT_DIR + "final_top1_accuracy_summary.md", "w") as file:
#     file.write(markdown_table)

# Display the markdown table
print(markdown_table)


|    | Architecture-Transformation   |   Original Code Top1-Acc |   Updated Code Top1-Acc |
|---:|:------------------------------|-------------------------:|------------------------:|
|  0 | RN50 flip CIFAR100            |                       98 |                      93 |
|  1 | RN50 rot90 CIFAR100           |                       97 |                      25 |
|  2 | RN101 flip CIFAR100           |                      nan |                     nan |
|  3 | RN101 rot90 CIFAR100          |                      nan |                     nan |
